# Data Aggregation and Group Operations

In [2]:
import pandas as pd
import numpy as np

## GroupBy mechanics

In [3]:
df = pd.DataFrame({'key1' : list('aabba'),
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5) * 20 + 10,
                   'data2' : np.random.randn(5) * 3 + 2})
df

,key1,key2,data1,data2
0,a,one,30.916193,3.707129
1,a,two,18.888989,3.564747
2,b,one,25.916005,2.612066
3,b,two,16.259812,-0.281612
4,a,one,12.224127,2.010411


In [4]:
df.groupby('key1')

In [5]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,20.676436,3.094096
b,21.087908,1.165227


In [8]:
double_grouped = df.groupby(['key1', 'key2']).mean()
double_grouped

data1     data2
key1 key2                     
a    one   21.570160  2.858770
     two   18.888989  3.564747
b    one   25.916005  2.612066
     two   16.259812 -0.281612

In [9]:
double_grouped.index

MultiIndex(levels=[['a', 'b'], ['one', 'two']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['key1', 'key2'])

In [12]:
double_grouped.loc[('a', 'one')]

data1    21.57016
data2     2.85877
Name: (a, one), dtype: float64

In [20]:
province = pd.Series(['M', 'M', 'B', 'V', 'V'], index=list('abcde'))

df.groupby(province).mean()

,data1,data2


In [21]:
province = pd.Series(['M', 'M', 'B', 'V', 'V'])

df.groupby(province).mean()

,data1,data2
B,25.916005,2.612066
M,24.902591,3.635938
V,14.241970,0.864399


In [22]:
df.groupby(province).size()

B    1
M    2
V    2
dtype: int64

In [24]:
df.groupby(province).std()

,data1,data2
B,NaN,NaN
M,8.504517,0.100679
V,2.853660,1.620705


### Iterating over groups

In [27]:
for group in df.groupby('key2'):
    print(type(group))

<class 'tuple'>
<class 'tuple'>


In [28]:
for id_, group in df.groupby('key2'):
    print(type(id_), type(group))

<class 'str'> <class 'pandas.core.frame.DataFrame'>
<class 'str'> <class 'pandas.core.frame.DataFrame'>


In [30]:
for id_, group in df.groupby('key2'):
    print(id_)
    print(group)

one
  key1 key2      data1     data2
0    a  one  30.916193  3.707129
2    b  one  25.916005  2.612066
4    a  one  12.224127  2.010411
two
  key1 key2      data1     data2
1    a  two  18.888989  3.564747
3    b  two  16.259812 -0.281612


### Selecting a column or subset of columns

In [34]:
df.groupby('key2')['data1'].sum()

key2
one    69.056325
two    35.148801
Name: data1, dtype: float64

## Data aggregation

In [37]:
df.groupby('key2').quantile(.9)

0.9,data1,data2
key2,,
one,29.916155,3.488116
two,18.626071,3.180111


In [40]:
gbobject = df.groupby('key2')
help(gbobject)

Help on DataFrameGroupBy in module pandas.core.groupby.groupby object:

class DataFrameGroupBy(NDFrameGroupBy)
 |  Class for grouping and aggregating relational data. See aggregate,
 |  transform, and apply functions on this object.
 |  
 |  It's easiest to use obj.groupby(...) to use GroupBy, but you can also do:
 |  
 |  ::
 |  
 |      grouped = groupby(obj, ...)
 |  
 |  Parameters
 |  ----------
 |  obj : pandas object
 |  axis : int, default 0
 |  level : int, default None
 |      Level of MultiIndex
 |  groupings : list of Grouping objects
 |      Most users should ignore this
 |  exclusions : array-like, optional
 |      List of columns to exclude
 |  name : string
 |      Most users should ignore this
 |  
 |  Notes
 |  -----
 |  After grouping, see aggregate, apply, and transform functions. Here are
 |  some other brief notes about usage. When grouping by multiple groups, the
 |  result index will be a MultiIndex (hierarchical) by default.
 |  
 |  Iteration produces (key, gr

In [43]:
def peak_to_peak(series):
    return series.max() - series.min()

gbobject.agg(peak_to_peak)

,data1,data2
key2,,
one,18.692065,1.696718
two,2.629177,3.846359


In [51]:
one_group = df[df['key2'] == 'one']
numeric_columns = one_group[['data1', 'data2']]
numeric_columns

,data1,data2
0,30.916193,3.707129
2,25.916005,2.612066
4,12.224127,2.010411


In [49]:
peak_to_peak(numeric_columns) 

data1    18.692065
data2     1.696718
dtype: float64

In [52]:
gbobject.agg(np.sum) 

,data1,data2
key2,,
one,69.056325,8.329606
two,35.148801,3.283135


In [53]:
gbobject.sum()

,data1,data2
key2,,
one,69.056325,8.329606
two,35.148801,3.283135


### Column-wise and multiple function application

In [54]:
!wget https://raw.githubusercontent.com/wesm/pydata-book/1st-edition/ch08/tips.csv

--2019-01-25 20:08:26--  https://raw.githubusercontent.com/wesm/pydata-book/1st-edition/ch08/tips.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.132.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.132.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943 (7,8K) [text/plain]
Saving to: ‘tips.csv’

tips.csv            100%[===================>]   7,76K  --.-KB/s    in 0s      

2019-01-25 20:08:27 (48,8 MB/s) - ‘tips.csv’ saved [7943/7943]



In [57]:
df = pd.read_csv('tips.csv')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [59]:
df.shape

(244, 7)

In [60]:
df.groupby('sex').sum()

,total_bill,tip,size
sex,,,
Female,1570.95,246.51,214
Male,3256.82,485.07,413


In [61]:
df.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,87,87,87,87,87,87
Male,157,157,157,157,157,157


In [62]:
df.groupby('sex').agg(['mean', 'std', 'sum'])

total_bill                          tip                        size  \
             mean       std      sum      mean       std     sum      mean   
sex                                                                          
Female  18.056897  8.009209  1570.95  2.833448  1.159495  246.51  2.459770   
Male    20.744076  9.246469  3256.82  3.089618  1.489102  485.07  2.630573   

                       
             std  sum  
sex                    
Female  0.937644  214  
Male    0.955997  413

In [65]:
stats = df.groupby('sex')[['total_bill', 'tip']].agg(['count', 'sum'])
stats

total_bill            tip        
            count      sum count     sum
sex                                     
Female         87  1570.95    87  246.51
Male          157  3256.82   157  485.07

In [73]:
tips = stats['tip']
tips['sum'] / tips['count']

sex
Female    2.833448
Male      3.089618
dtype: float64

In [75]:
df.groupby('sex')[['total_bill', 'tip']].agg(['count', 'sum', peak_to_peak, lambda s: s[-1:]]) 

total_bill                                  tip                       \
            count      sum peak_to_peak <lambda> count     sum peak_to_peak   
sex                                                                           
Female         87  1570.95        41.23    18.78    87  246.51          5.5   
Male          157  3256.82        43.56    17.82   157  485.07          9.0   

                 
       <lambda>  
sex              
Female     3.00  
Male       1.75

In [76]:
df.groupby('sex')[['total_bill', 'tip']].agg([('media', 'mean'), ('desviacion', np.std)])

total_bill                  tip           
            media desviacion     media desviacion
sex                                              
Female  18.056897   8.009209  2.833448   1.159495
Male    20.744076   9.246469  3.089618   1.489102

## Group-wise operations and transformations

In [82]:
df.groupby('sex')[['total_bill', 'tip']].mean().add_prefix('avg_')

,avg_total_bill,avg_tip
sex,,
Female,18.056897,2.833448
Male,20.744076,3.089618


### Apply: General split-apply-combine

In [85]:
gbobject

In [88]:
gbobject.apply(np.sum)

,data1,data2,key1
key2,,,
one,69.056325,8.329606,aba
two,35.148801,3.283135,ab


In [102]:
def top_5(group):
    print(type(group), group.shape)
    return group.sort_values(by='tip', ascending=False).iloc[:5]
    
top_5(df)

<class 'pandas.core.frame.DataFrame'> (244, 7)


,total_bill,tip,sex,smoker,day,time,size
170,50.81,10.00,Male,Yes,Sat,Dinner,3
212,48.33,9.00,Male,No,Sat,Dinner,4
23,39.42,7.58,Male,No,Sat,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
141,34.30,6.70,Male,No,Thur,Lunch,6


In [103]:
df.groupby('smoker').apply(top_5)

<class 'pandas.core.frame.DataFrame'> (151, 7)
<class 'pandas.core.frame.DataFrame'> (151, 7)
<class 'pandas.core.frame.DataFrame'> (93, 7)


total_bill    tip     sex smoker   day    time  size
smoker                                                          
No     212       48.33   9.00    Male     No   Sat  Dinner     4
       23        39.42   7.58    Male     No   Sat  Dinner     4
       59        48.27   6.73    Male     No   Sat  Dinner     4
       141       34.30   6.70    Male     No  Thur   Lunch     6
       47        32.40   6.00    Male     No   Sun  Dinner     4
Yes    170       50.81  10.00    Male    Yes   Sat  Dinner     3
       183       23.17   6.50    Male    Yes   Sun  Dinner     4
       214       28.17   6.50  Female    Yes   Sat  Dinner     3
       181       23.33   5.65    Male    Yes   Sun  Dinner     2
       211       25.89   5.16    Male    Yes   Sat  Dinner     4

#### Suppressing the group keys

In [104]:
df.groupby('smoker', group_keys=False).apply(top_5)

<class 'pandas.core.frame.DataFrame'> (151, 7)
<class 'pandas.core.frame.DataFrame'> (151, 7)
<class 'pandas.core.frame.DataFrame'> (93, 7)


,total_bill,tip,sex,smoker,day,time,size
212,48.33,9.00,Male,No,Sat,Dinner,4
23,39.42,7.58,Male,No,Sat,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
141,34.30,6.70,Male,No,Thur,Lunch,6
47,32.40,6.00,Male,No,Sun,Dinner,4
170,50.81,10.00,Male,Yes,Sat,Dinner,3
183,23.17,6.50,Male,Yes,Sun,Dinner,4
214,28.17,6.50,Female,Yes,Sat,Dinner,3
181,23.33,5.65,Male,Yes,Sun,Dinner,2
211,25.89,5.16,Male,Yes,Sat,Dinner,4


### Quantile and bucket analysis

In [107]:
pd.cut(df['tip'], 5).value_counts()

(0.991, 2.8]    120
(2.8, 4.6]       93
(4.6, 6.4]       24
(6.4, 8.2]        5
(8.2, 10.0]       2
Name: tip, dtype: int64

In [110]:
pd.qcut(df['tip'], 4).value_counts()

(0.999, 2.0]     78
(3.562, 10.0]    61
(2.9, 3.562]     61
(2.0, 2.9]       44
Name: tip, dtype: int64

### Example: Filling missing values with group-specific values

In [119]:
with_holes = df.copy()
with_holes['tip'][::3] = np.nan
with_holes['total_bill'][::4] = np.nan
with_holes.head()

/home/dani/anaconda3/envs/master2019/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dani/anaconda3/envs/master2019/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,total_bill,tip,sex,smoker,day,time,size
0,NaN,NaN,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,NaN,Male,No,Sun,Dinner,2
4,NaN,3.61,Female,No,Sun,Dinner,4


In [129]:
guesses.loc['No']

tip            3.180882
total_bill    17.810000
Name: No, dtype: float64

In [132]:
with_holes.groupby('smoker').apply(lambda group: group.fillna(group.mean()))

total_bill       tip     sex smoker   day    time  size
smoker                                                             
No     0     18.851304  3.180882  Female     No   Sun  Dinner     2
       1     10.340000  1.660000    Male     No   Sun  Dinner     3
       2     21.010000  3.500000    Male     No   Sun  Dinner     3
       3     23.680000  3.180882    Male     No   Sun  Dinner     2
       4     18.851304  3.610000  Female     No   Sun  Dinner     4
       5     25.290000  4.710000    Male     No   Sun  Dinner     4
       6      8.770000  3.180882    Male     No   Sun  Dinner     2
       7     26.880000  3.120000    Male     No   Sun  Dinner     4
       8     18.851304  1.960000    Male     No   Sun  Dinner     2
       9     14.780000  3.180882    Male     No   Sun  Dinner     2
       10    10.270000  1.710000    Male     No   Sun  Dinner     2
       11    35.260000  5.000000  Female     No   Sun  Dinner     4
       12    18.851304  3.180882    Male     No   Sun  Dinner     2
       13    18.430000  3.000000    Male     No   Sun  Dinner     4
       14    14.830000  3.020000  Female     No   Sun  Dinner     2
       15    21.580000  3.180882    Male     No   Sun  Dinner     2
       16    18.851304  1.670000  Female     No   Sun  Dinner     3
       17    16.290000  3.710000    Male     No   Sun  Dinner     3
       18    16.970000  3.180882  Female     No   Sun  Dinner     3
       19    20.650000  3.350000    Male     No   Sat  Dinner     3
       20    18.851304  4.080000    Male     No   Sat  Dinner     2
       21    20.290000  3.180882  Female     No   Sat  Dinner     2
       22    15.770000  2.230000  Female     No   Sat  Dinner     2
       23    39.420000  7.580000    Male     No   Sat  Dinner     4
       24    18.851304  3.180882    Male     No   Sat  Dinner     2
       25    17.810000  2.340000    Male     No   Sat  Dinner     4
       26    13.370000  2.000000    Male     No   Sat  Dinner     2
       27    12.690000  3.180882    Male     No   Sat  Dinner     2
       28    18.851304  4.300000    Male     No   Sat  Dinner     2
       29    19.650000  3.000000  Female     No   Sat  Dinner     2
...                ...       ...     ...    ...   ...     ...   ...
Yes    203   16.400000  2.500000  Female    Yes  Thur   Lunch     2
       204   20.897794  3.066167    Male    Yes  Thur   Lunch     4
       205   16.470000  3.230000  Female    Yes  Thur   Lunch     3
       206   26.590000  3.410000    Male    Yes   Sat  Dinner     3
       207   38.730000  3.066167    Male    Yes   Sat  Dinner     4
       208   20.897794  2.030000    Male    Yes   Sat  Dinner     2
       209   12.760000  2.230000  Female    Yes   Sat  Dinner     2
       210   30.060000  3.066167    Male    Yes   Sat  Dinner     3
       211   25.890000  5.160000    Male    Yes   Sat  Dinner     4
       213   13.270000  3.066167  Female    Yes   Sat  Dinner     2
       214   28.170000  6.500000  Female    Yes   Sat  Dinner     3
       215   12.900000  1.100000  Female    Yes   Sat  Dinner     2
       216   20.897794  3.066167    Male    Yes   Sat  Dinner     5
       217   11.590000  1.500000    Male    Yes   Sat  Dinner     2
       218    7.740000  1.440000    Male    Yes   Sat  Dinner     2
       219   30.140000  3.066167  Female    Yes   Sat  Dinner     4
       220   20.897794  2.200000    Male    Yes   Fri   Lunch     2
       221   13.420000  3.480000  Female    Yes   Fri   Lunch     2
       222    8.580000  3.066167    Male    Yes   Fri   Lunch     1
       224   20.897794  1.580000    Male    Yes   Fri   Lunch     2
       225   16.270000  3.066167  Female    Yes   Fri   Lunch     2
       226   10.090000  2.000000  Female    Yes   Fri   Lunch     2
       229   22.120000  2.880000  Female    Yes   Sat  Dinner     2
       230   24.010000  2.000000    Male    Yes   Sat  Dinner     4
       231   15.690000  3.066167    Male    Yes   Sat  Dinner     3
       234   15.530000  3.066167    Male    Yes   Sat  Dinner     2


## Pivot tables and Cross-tabulation

In [139]:
df.pivot_table(index='day', columns='time', aggfunc='count') 

sex         size       smoker          tip       total_bill      
time Dinner Lunch Dinner Lunch Dinner Lunch Dinner Lunch     Dinner Lunch
day                                                                      
Fri    12.0   7.0   12.0   7.0   12.0   7.0   12.0   7.0       12.0   7.0
Sat    87.0   NaN   87.0   NaN   87.0   NaN   87.0   NaN       87.0   NaN
Sun    76.0   NaN   76.0   NaN   76.0   NaN   76.0   NaN       76.0   NaN
Thur    1.0  61.0    1.0  61.0    1.0  61.0    1.0  61.0        1.0  61.0

In [143]:
df.pivot_table(index = 'sex', columns='smoker', values=['total_bill', 'tip'])

tip           total_bill           
smoker        No       Yes         No        Yes
sex                                             
Female  2.773519  2.931515  18.105185  17.977879
Male    3.113402  3.051167  19.791237  22.284500

In [144]:
df.pivot_table(index = 'sex', columns='smoker', values='tip', aggfunc='mean')

smoker,No,Yes
sex,,
Female,2.773519,2.931515
Male,3.113402,3.051167
